<a href="https://colab.research.google.com/github/rmfpafls/Study_BeautifulSoup_discord_bot/blob/master/How_To_Recommend_Anything.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to recommend Anything / Deep Recommender

관련 자료

: https://www.kaggle.com/code/morrisb/how-to-recommend-anything-deep-recommender/notebook




데이터셋

: (Netflix Prize data) https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data

: (The Movies Dataset) https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset

# 1. Import Libaries


In [ ]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357268 sha256=5e9ec25c1658d680ebfbfba444e5ab74c8014119355e1838da32cd37ed7593cb
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
pip install keras


In [ ]:
pip install LightFM

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for LightFM: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808328 sha256=631a1257779c43e1ceb33916f95de1a983244330c8aeff3b8d5fcaae15f6c430
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built LightFM


In [ ]:
# 데이터를 저장하기 위해
import pandas as pd

# 선형대수를 수행하기 위해
import numpy as np

# 그래프를 생성하기 위해
import matplotlib.pyplot as plt

# 인터랙티브 그래프를 생성하기 위해
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# 리스트를 이동시키기 위해
from collections import deque

# 벡터 간의 유사성을 계산하기 위해
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# 추천 시스템을 사용하기 위해
import surprise as sp
from surprise.model_selection import cross_validate

# 딥러닝 모델을 생성하기 위해
from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from keras.models import Model

# 희소 행렬을 생성하기 위해
from scipy.sparse import coo_matrix

# LightFM을 사용하기 위해
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

# 희소 행렬을 쌓기 위해
from scipy.sparse import vstack


# 2. Load Movie-Data


In [ ]:
# 1. 모든 영화 데이터 뽑아오기
movie_titles = pd.read_csv('/content/drive/MyDrive/How to recommend anything/movie_titles.csv',
                           encoding = 'ISO-8859-1', # 주로 유럽 언어에서 사용되는 인코딩 방식
                           header = None,  # csv 파일에 열 이름이 포함되어 있지 않음을 나타냄
                           names = ['Id', 'Year', 'Name'], # names인자는 header = none일 때 사용되며, 데이터의 프레임의 열 이름을 지정함.
                           on_bad_lines= 'skip').set_index('Id')  # on_bad_lines
                           # csv 파일을 읽어서 열 이름을 지정하고, 특정 열을 인덱스로 설정하는 과정

print('Shape Movie-Titles:\t{}'.format(movie_titles.shape))
movie_titles.sample(5) # 랜덤으로 5개 추출하기

Shape Movie-Titles:	(17434, 2)


,Year,Name
Id,,
2169,1977.0,Orca: The Killer Whale
5594,1978.0,Toolbox Murders
17126,1995.0,Kitaro: An Enchanted Evening
5693,1977.0,Doctor Who: The Talons of Weng-Chiang
5616,1966.0,The Blue Max


In [ ]:
# movie metadata 의 데이터 가지고 오기
movie_metadata = pd.read_csv('/content/drive/MyDrive/How to recommend anything/The movies Dataset/movies_metadata.csv', low_memory=False)[['original_title', 'overview', 'vote_count']].set_index('original_title').dropna()
# 평점이 적은 영화 필터링하고, 'vote_count'열 제거
movie_metadata = movie_metadata[movie_metadata['vote_count']>10].drop('vote_count', axis=1)

print('Shape Movie-Metadata:\t{}'.format(movie_metadata.shape))
# movie_metadata.shape : 데이터 프레임의 행과 열의 수를 튜플 형태로 변환
movie_metadata.sample(5) #무작위로 5개의 샘플 데이터 출력

Shape Movie-Metadata:	(21604, 1)


,overview
original_title,
Børning,For all who remember Cannonball Run. Here is a...
Head Over Heels,"After many years of marriage, Walter and Madge..."
킬러들의 수다,Guns &amp; Talks is a South Korean film releas...
さよなら歌舞伎町,This erotically charged drama from Japanese di...
Enemy,A mild-mannered college professor discovers a ...


# 3. Load User-Data And Preprocess Data-Structure

In [ ]:
# 1. 파일 불러오기
df_raw = pd.read_csv('/content/drive/MyDrive/How to recommend anything/combined_data_1.txt/combined_data_1.txt', header=None, names=['User', 'Rating', 'Date'], usecols=[0, 1, 2])

# 2. 빈 줄을 찾아 데이터프레임을 분할하는 기준 설정
tmp_movies = df_raw[df_raw['Rating'].isna()]['User'].reset_index()
# isna() : Pandas에서 제공하는 함수 isnull()과 같은 기능
# df_raw[df_raw['Rating'].isna()] : Rating 열에서 값이 없는 행을 찾는다.
# ['User'] : 빈 줄에서 user열을 추출한다.
#reset_index() : 인덱스를 초기화하여 새로운 데이터 프레임을 만든다.

movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]


# 3. 영화 시작 위치를 이동시켜 시작 및 종료 지점 설정
##### 이게 뭔 소리람
shifted_movie_indices = deque(movie_indices)
# deque() : movie_indices리스트를 덱으로 변환, collections 모듈에서 제공하는 양방향 큐 자료구조
# 양방향으로 데이터를 추가/삭제, append, pop을 빠르게 수행할 수 있어서 빠르게 동작가능
shifted_movie_indices.rotate(-1)
# 덱을 왼쪽으로 한칸씩 회전시켜 각 영화의 종료 지점을 설정


# 4. 데이터를 저장할 리스트 준비
user_data = []

# 5. 각 영화에 대해 반복 작업 수행
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):

    # 5-1. 마지막 영화인지 확인하고 데이터 추출
    if df_id_1<df_id_2: # 영화가 마지막 영화인지 확인
        tmp_df = df_raw.loc[df_id_1+1:df_id_2-1].copy()
        # 현재 영화에 해당하는 평점 데이터를 추출. Id가 있는 줄을 제외하고 실제 평점 데이터를 가져오도록한다.
        # copy() : 원본 데이터프레임에서 슬라이스한 데이터를 복사하여 tmp_df에 저장
    else:
        tmp_df = df_raw.loc[df_id_1+1:].copy()

    # 5-2 영화 id 열 추가 및 데이터 저장
    tmp_df['Movie'] = movie_id
    user_data.append(tmp_df)

# 6. 모든 데이터를 하나의 데이터프레임으로 결합
df = pd.concat(user_data)

# 7. 사용하지 않는 변수 삭제 및 데이터 확인
del user_data, df_raw, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
# del : 메모리를 절약하기 위해 사용이 끝난 변수들을 삭제
print('Shape User-Ratings:\t{}'.format(df.shape))
df.sample(5)

Shape User-Ratings:	(24053764, 4)


,User,Rating,Date,Movie
1844018,707496,4.0,2005-10-01,334
12476368,2138429,3.0,2004-07-11,2391
22105666,394483,1.0,2003-05-13,4159
14353971,2266650,3.0,2004-12-16,2780
1013284,2351296,5.0,2005-11-14,239
